# Gaussian Splatting

Repo: https://github.com/camenduru/gaussian-splatting
(https://github.com/graphdeco-inria/gaussian-splatting)

Info: https://www.plainconcepts.com/3d-gaussian-splatting/
https://danielpikl.medium.com/what-is-the-difference-between-nerf-and-gaussian-splatting-technology-d6d9848faf80

Guide: https://medium.com/@adrian.milla/rebuild-your-home-in-3d-with-gaussian-splatting-f6d7222717fb

Video reference: https://www.youtube.com/watch?v=ZLTkMF22BaA


### 0. Installation and dependencies

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

In [ ]:
!pip install ffmpeg

In [ ]:
!pip install opencv-python-headless

In [ ]:
# colmap > 3.6 needed (used 3.9)
!sudo apt-get install colmap

In [ ]:
# GET DEMO FILES
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

In [ ]:
# GET OUR FILES
!cp -r /content/drive/MyDrive/GaussianSplatting/Catedral.zip /content/gaussian-splatting/
!unzip Catedral.zip

### 1. Trim video into images with ffmpeg

In [ ]:
!mkdir /content/Catedral/
!mkdir /content/Catedral/input/
%cd /content/Catedral/input/

/content/Catedral/input


In [ ]:
!ffmpeg -i /content/Catedral_trim.mp4 -r 2 /content/Catedral/input/image%03d.jpg

In [ ]:
%cd /content/gaussian-splatting

/content/gaussian-splatting


### 2. Get cloud of points with COLMAP

In [ ]:
!python convert.py -s /content/Catedral/

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: eglfs, linuxfb, minimal, minimalegl, offscreen, vnc, xcb.

*** Aborted at 1703998894 (unix time) try "date -d @1703998894" if you are using GNU date ***
PC: @                0x0 (unknown)
*** SIGABRT (@0x1c04) received by PID 7172 (TID 0x7f4d871d9080) from PID 7172; stack trace: ***
    @     0x7f4d8f789046 (unknown)
    @     0x7f4d8d809520 (unknown)
    @     0x7f4d8d85d9fc pthread_kill
    @     0x7f4d8d809476 raise
    @     0x7f4d8d7ef7f3 abort
    @     0x7f4d8dd92ba3 QMessageLogger::fatal()
    @     0x7f4d8e399713 QGuiApplicationPrivate::createPlatformIntegration()
    @     0x7f4d8e399c08 QGuiApplicationPrivate::createEventDispatcher()
    @     0x7f4d8dfc2b17 QCoreApplicationPriv

### 3. Train GAUSSIAN SPLATING to obtain 3D model

In [ ]:
!python train.py -s /content/gaussian-splatting/Catedral

2024-02-01 16:26:30.879312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 16:26:30.879371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 16:26:30.881254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 16:26:30.892547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-01 16:26:32.379212: W tensorflow/comp

### 4. Save output and visualize *.ply*





In [ ]:
!zip -r /content/gaussian-splatting_out.zip /content/gaussian-splatting/output/

  adding: content/gaussian-splatting/output/ (stored 0%)
  adding: content/gaussian-splatting/output/ee7508cc-0/ (stored 0%)
  adding: content/gaussian-splatting/output/ee7508cc-0/point_cloud/ (stored 0%)
  adding: content/gaussian-splatting/output/ee7508cc-0/point_cloud/iteration_7000/ (stored 0%)
  adding: content/gaussian-splatting/output/ee7508cc-0/point_cloud/iteration_7000/point_cloud.ply (deflated 11%)
  adding: content/gaussian-splatting/output/ee7508cc-0/point_cloud/iteration_30000/ (stored 0%)
  adding: content/gaussian-splatting/output/ee7508cc-0/point_cloud/iteration_30000/point_cloud.ply (deflated 11%)
  adding: content/gaussian-splatting/output/ee7508cc-0/cameras.json (deflated 67%)
  adding: content/gaussian-splatting/output/ee7508cc-0/input.ply (deflated 44%)
  adding: content/gaussian-splatting/output/ee7508cc-0/events.out.tfevents.1706804793.7c43c954c3e4.5070.0 (deflated 15%)
  adding: content/gaussian-splatting/output/ee7508cc-0/cfg_args (deflated 22%)


In [ ]:
!git clone https://github.com/camenduru/splat
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O /content/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd /content/splat
!python -m http.server 7860